# Compile constituency parse info

(one-time)

In [1]:
import sys, os, json, string
import spacy
import numpy as np
import pandas as pd
from collections import defaultdict
from benepar.spacy_plugin import BeneparComponent

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(BeneparComponent('benepar_en'))

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
with open('data/moby-passages-36/item_to_passage_final.json') as f:
    item_to_passage_cleaned = json.load(f)
item_to_doc = {}
for item, passage in item_to_passage_cleaned.items():
#     print('processing', item)
    __doc = nlp(passage)
    item_to_doc[item] = __doc

In [3]:
item_to_recstring_list = {}
with open('data/moby-passages-36/passages-with-line-break-and-recstring.tsv') as f:
    next(f)
    for line in f:
        _, _, item, _, recstring = line.strip().split('\t')
        item_to_recstring_list[item] = recstring.split()

In [4]:
item_to_doc_id_to_recstring_id = {}
for item, doc in item_to_doc.items():
#     print('\n\n\n', item)
    doc_id_to_recstring = {}
    curr_idx = 0
#     matched = 0
    for idx, x in enumerate(doc):
        if all(y in string.punctuation for y in str(x)):
            continue
        elif ((str(x)[0] not in {'"'} and any(y in string.punctuation for y in str(x)))\
                and (str(x) not in {'Mrs.', 'Ms.', 'Dr.'}))\
                or (str(x) == 'not' and str(doc[idx - 1]) == 'can'):
            assert curr_idx > 0
            curr_idx -= 1
        doc_id_to_recstring[x.i] = curr_idx
        
#         if str(x).lower() == item_to_recstring_list[item][curr_idx]:
#             matched += 1
        
        curr_idx += 1
    assert 1 + max(v for k, v in doc_id_to_recstring.items()) == len(item_to_recstring_list[item])
#     print(item, 'matched', matched / len(item_to_recstring_list[item]))
    item_to_doc_id_to_recstring_id[item] = doc_id_to_recstring

In [5]:
def bottom_up_representation(sent):
    constituent_to_depth = {}
    token_to_parents = {}
    for token in sent:
        curr = token
        depth = 1
        parents = set()
        while curr._.parent:
            curr = curr._.parent
            parents.add(curr)
            if curr not in constituent_to_depth:
                constituent_to_depth[curr] = depth
            constituent_to_depth[curr] = max(depth, constituent_to_depth[curr])
            depth += 1
        token_to_parents[token] = parents
    return constituent_to_depth, token_to_parents

def bottom_up_feature_vector(sent, k=1):
    rv = [0]
    constituent_to_depth, token_to_parents = bottom_up_representation(sent)
    for idx, token in enumerate(sent):
        if idx == 0:
            continue
        prev_token = sent[idx - 1]
        
        common_ancestors = token_to_parents[token] & token_to_parents[prev_token]
        common_ancestors_depths = set(constituent_to_depth[anc] for anc in common_ancestors)
        
        if k in common_ancestors_depths:
            rv.append(1)
        else:
            rv.append(0)
    return rv

def pretty_print_sisters_bottom_up(sent, k):
    x = bottom_up_feature_vector(sent, k)
    print(x)
    for idx, val in enumerate(x):
        if val == 0:
            print('\n' + str(sent[idx]), end=' ')
        else:
            print(sent[idx], end=' ')

In [6]:
def get_sentence_depth(sentence):
    k = 20
    while True:
        feature_vector = bottom_up_feature_vector(sentence, k)
        assert sum(feature_vector) in {0, len(sentence) - 1}
        if sum(feature_vector) == len(sentence) - 1:
            return k
        k -= 1

In [7]:
# wow this logic was tricky

MAX_DEPTH = 30

item_to_level_to_constituency_ids = {}

for item, doc in item_to_doc.items():
    level_to_constituency_ids = defaultdict(list)
    sentences = list(doc.sents)
    recstring = item_to_recstring_list[item]
    doc_id_to_recstring_id = item_to_doc_id_to_recstring_id[item]
#     print(doc_id_to_recstring_id)
    for sentence in sentences:
        sentence_depth = get_sentence_depth(sentence)
        
        for k in range(1, sentence_depth + 1):
            feature_vector = list(reversed(bottom_up_feature_vector(sentence, k)))
            reversed_sentence = reversed(sentence)
            token = next(reversed_sentence)
            prev = feature_vector[0]
            if prev == 0:
                clusters = [[]]
            else:
                clusters = [[token]]
            
            for val in feature_vector[1:]:
                token = next(reversed_sentence)
                if val == 1:
                    clusters[-1].append(token)
                if val == 0:
                    if prev == 1:
                        clusters[-1].append(token)
                    clusters.append([])
                prev = val
            clusters = [x[::-1] for x in clusters if x][::-1]
            assert all(len(x) > 1 for x in clusters)
            
            for cluster in clusters:
                if not str(cluster[0])[0].isalpha():
                    cluster = cluster[1:]
                
                cluster_ids = [token.i for token in cluster]
#                 selected_recstring_tokens = [recstring[item_to_doc_id_to_recstring_id[item][i]] for i in cluster_ids]
                recstring_ids = [doc_id_to_recstring_id[i] for i in cluster_ids if i in doc_id_to_recstring_id]
#                 recstring_tokens = [recstring[i] for i in recstring_ids]
                if len(recstring_ids) > 1:
                    level_to_constituency_ids[sentence_depth].append(recstring_ids)

#         break
    item_to_level_to_constituency_ids[item] = level_to_constituency_ids
#     break

In [8]:
with open('data/moby-passages-36/item_to_level_to_constituency_ids.json', 'w') as f:
    json.dump(item_to_level_to_constituency_ids, f, indent=1)